In [ ]:
import math
import os
import os
import pickle
import random
import shutil
import time

import keras
import moviepy.video.io.ImageSequenceClip
import numpy as np
import pandas as pd
import pptk
import tensorflow as tf
from keras import backend as K
from keras.backend import clear_session
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers
from tqdm.notebook import tqdm
print('Tensorflow: ', tf.__version__)
from keras import layers
import datetime
from PIL import Image
import glob
from keras.optimizers import Adam
import tensorflow_addons as tfa
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    RepeatedKFold,
    StratifiedKFold,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from keras.optimizers import Adam
from sklearn.model_selection import ParameterGrid



# Before instantiating a tf.data.Dataset obj & before model creation, call:
clear_session()

# Hyperparameter and architecture search for a deep variational autoencoder on 3d point cloud/mesh/vertex data
# Published July 2022. S.H.
# Network capable of learning and generating 3d mesh structures
# Pass examples as 1d vectors.
# Multiple related meshes may be passed as concatenated vectors [xyzxyz] + [xyzx]. Network learns relation.
# network dimensions collapse to a latent vector, decending width.
# to tweak/grid search:
# optimal latent vector size may vary
# optimal layer depth and compression dimension varies depending on input complexity



In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
def calc_closest_factors(c: int):
    """Calculate the closest two factors of c.
    
    Returns:
      [int, int]: The two factors of c that are closest; in other words, the
        closest two integers for which a*b=c. If c is a perfect square, the
        result will be [sqrt(c), sqrt(c)]; if c is a prime number, the result
        will be [1, c]. The first number will always be the smallest, if they
        are not equal.
    """    
    if c//1 != c:
        raise TypeError("c must be an integer.")

    a, b, i = 1, c, 0
    while a < b:
        i += 1
        if c % i == 0:
            a = i
            b = c//a
    
    return [b, a]

In [ ]:
# Clear previous tensorboard data
try:
    shutil.rmtree("M:\\keras_env\\train")
except:
    pass

In [ ]:
try:
    shutil.rmtree("M:\\keras_env\\validation")
except:
    pass

In [ ]:
#Read RAW data
#Read in 1d data arrays for each mesh/point cloud in the form[x1,y1,z1, x2,y2,z2.... xn, yn, zn]
#Each row of the data frame/matrix should be one example.
df = pd.read_csv('1d_xyz_vertex_point_cloud_data.csv')

In [ ]:

df.drop('Unnamed: 0', inplace = True, axis=1)
#df.drop('index', inplace = True, axis=1)

In [ ]:
#Add additional data, for more samples, boot strap.
merges = pd.read_csv("merges.csv")

In [ ]:
merges.drop('Unnamed: 0', inplace = True, axis=1)

In [ ]:
df = df.append(merges)#.copy()

In [ ]:
# open a file, where you stored the pickled data
file = open('mask_array', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [ ]:
mask_array = data

In [ ]:
mask_array = [str(x) for x in mask_array]

In [ ]:
apply_mask = False

In [ ]:
if(apply_mask):
    df = df[mask_array].copy()
    
else:
    mask_array = np.ones(len(df.iloc[0]))

In [ ]:
#Scale for NN, inverse scale later
scaler = StandardScaler()

In [ ]:
#scaler = MinMaxScaler()

In [ ]:
#scaler = Normalizer()

In [ ]:
scaler.fit(df)

In [ ]:
scale = True

In [ ]:
if(scale):
    df = scaler.transform(df)

In [ ]:
x_train, x_test, = train_test_split(
     df, test_size=0.2, random_state=0)

In [ ]:
#1d array of xyzxyzxyz...

original_dim = 11496
intermediate_dim = 64
latent_dim = 2

In [ ]:
#These layers can be changed
original_dim = df.shape[1]

scale_factor = 12

intermediate_dim = int(original_dim/scale_factor)

deep_dim_one = int(intermediate_dim/2)

latent_dim = 2

In [ ]:
print(intermediate_dim,deep_dim_one,int(deep_dim_one/2), int(deep_dim_one/4), latent_dim)

In [ ]:
kernel_regularizer_val = 0.000001
activity_regularizer = 0.0

inputs = keras.Input(shape=(original_dim,))

h = layers.Dense(intermediate_dim, activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                activity_regularizer=regularizers.L2(activity_regularizer))(inputs)

h1 = layers.Dense(deep_dim_one, activation='relu')(h)

z_mean = layers.Dense(latent_dim)(h1)

z_log_sigma = layers.Dense(latent_dim)(h1)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(int(deep_dim_one/10),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(latent_inputs)

outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])

vae = keras.Model(inputs, outputs, name='vae_mlp')

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(int(deep_dim_one/10),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(latent_inputs)

x1 = layers.Dense(int(deep_dim_one/8),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x)

x2 = layers.Dense(int(deep_dim_one/6),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x1)

x3 = layers.Dense(int(deep_dim_one/4),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x2)

x4 = layers.Dense(int(deep_dim_one/2),
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x3)

x5 = layers.Dense(deep_dim_one,
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x4)

x6 = layers.Dense(intermediate_dim,
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(x5)

#x =  layers.Dropout(0.02)(x) #exp

outputs = layers.Dense(original_dim, activation='sigmoid')(x6)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])

vae = keras.Model(inputs, outputs, name='vae_mlp')


In [ ]:
#encoder

print(intermediate_dim,
      deep_dim_one,
      deep_dim_one/2,
      int(deep_dim_one/4),
      int(deep_dim_one/6),
      int(deep_dim_one/8),
      int(deep_dim_one/10))

In [ ]:
#decoder

print( int(deep_dim_one/10),
      int(deep_dim_one/8),
      int(deep_dim_one/6),
      int(deep_dim_one/4),
      int(deep_dim_one/2),
      int(deep_dim_one),
      int(intermediate_dim))


In [ ]:
#sgd = optimizers.SGD(lr=0.01, clipnorm=1.)


In [ ]:
initial_learning_rate = 0.0007

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96, #96
    staircase=True)

In [ ]:
momentum = 0.8

In [ ]:
opt_adam = Adam(clipnorm=0.01,
                learning_rate = 0.002) #lr_schedule) #0.000005)
#opt_adam = Adam(clipvalue=0.5)

#opt_adam = Adam(clipvalue=0.5,clipnorm=0.01)


In [ ]:
opt_adam.get_config()

#reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
reconstruction_loss = loss_fn(inputs, outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer=opt_adam)

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
#print(vae.summary())

In [ ]:
#plot_model(vae, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
x_train.shape

In [ ]:
from keras.callbacks import Callback

class stopAtLossValue(Callback):

        def on_batch_end(self, batch, logs={}):
            THR = 1000.03 #Assign THR with the value at which you want to stop training.
            THR_L = -THR
            if logs.get('loss') <= THR and logs.get('loss') >= THR_L:
                 self.model.stop_training = True

In [ ]:
original_dim = df.shape[1]

scale_factor = 12

intermediate_dim = int(original_dim/scale_factor)

deep_dim_one = int(intermediate_dim/2)

latent_dim = 2

In [ ]:
int(deep_dim_one/(0+1))


In [ ]:
def create_model(
    layers_param=1,
    kernel_reg=tf.keras.regularizers.l1_l2(l1=0, l2=0),
    width=15,
    learning_rate=0.01,
    dropout_val=0.2,
    input_dim_val=df.shape[1],
    scale_factor = 12
):
    
    original_dim = df.shape[1]

    #scale_factor = 12

    intermediate_dim = int(original_dim/scale_factor)

    deep_dim_one = int(intermediate_dim/2)

    latent_dim = 2
    
    
    inputs = keras.Input(shape=(original_dim,))

    h = layers.Dense(intermediate_dim, activation='relu',
                     kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                    activity_regularizer=regularizers.L2(activity_regularizer))(inputs)

    h1 = layers.Dense(deep_dim_one, activation='relu')(h)
    
    live_h = h1
    for i in range(0, layers_param):
        print(int(deep_dim_one/(i+1)))
        h2 = layers.Dense(int(deep_dim_one/(i+1)), activation='relu')(live_h)
        live_h = h2

    z_mean = layers.Dense(latent_dim)(h2)

    z_log_sigma = layers.Dense(latent_dim)(h2)

    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                                  mean=0., stddev=0.1)
        return z_mean + K.exp(z_log_sigma) * epsilon

    z = layers.Lambda(sampling)([z_mean, z_log_sigma])

    # Create encoder
    encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

    # Create decoder
    latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
    x = layers.Dense(int(deep_dim_one/10),
                     #kernel_initializer='ones',
                     activation='relu',
                     kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                     activity_regularizer=regularizers.L2(activity_regularizer)

                    )(latent_inputs)
    
    live_h = x
    for j in range(layers_param+1,0, -1):
        print(int(deep_dim_one/(j+1)))
        x1 = layers.Dense(int(deep_dim_one/(j+1)), #flip? reverse..?
                 #kernel_initializer='ones',
                 activation='relu',
                 kernel_regularizer=regularizers.L1(kernel_regularizer_val),
                 activity_regularizer=regularizers.L2(activity_regularizer)
                 
                )(live_h)
        live_h = x1
    

    outputs = layers.Dense(original_dim, activation='sigmoid')(x1)
    decoder = keras.Model(latent_inputs, outputs, name='decoder')

    # instantiate VAE model
    outputs = decoder(encoder(inputs)[2])

    vae = keras.Model(inputs, outputs, name='vae_mlp')
    #return inputs, outputs
    loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
    reconstruction_loss = loss_fn(inputs, outputs)

    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer=opt_adam)
    return vae

In [ ]:
metric = tfa.metrics.r_square.RSquare()

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
opt_adam = Adam(clipnorm=0.01,
                learning_rate = 0.002)

In [ ]:
parameters = {'learning_rate': [0.0001, 0.001, 0.00001, 0.00005],
             'layers_param': [2, 3, 4],
             'scale_factor':[12, 24, 36, 48, 60]}

In [ ]:
kmodel = KerasRegressor(
        build_fn=create_model, verbose=0, learning_rate=0.0001, layers_param=1, width=1
    )

grid = GridSearchCV(
    kmodel, parameters, verbose=3, cv=[(slice(None), slice(None))]
)  # Negate CV in param search for speed

pg = ParameterGrid(parameters)
pg = len(pg)
print(pg)

In [ ]:
int(df.shape[1]/40)

In [ ]:
grid.fit(x_train,
        x_train,
         batch_size = int(df.shape[1]/10),
         epochs=300,
        callbacks=[TensorBoard(log_dir='M:\\keras_env'),
                  tf.keras.callbacks.TerminateOnNaN(),
                  tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', verbose=1, patience=400, restore_best_weights=True),
                  stopAtLossValue(),
                  tf.keras.callbacks.ModelCheckpoint("M:\\kerascheckpoints", monitor='loss', save_best_only=False,
                 mode='auto', save_freq=5000)])

In [ ]:
grid.best_params_

In [ ]:
estimator = KerasRegressor(
        build_fn=create_model, verbose=0, learning_rate=0.0001, layers_param=3, width=1, scale_factor = 60
    )

In [ ]:
estimator.fit(x_train,
        x_train,
        batch_size = int(df.shape[1]/20),
         epochs=300,
        callbacks=[TensorBoard(log_dir='M:\\keras_env'),
                  tf.keras.callbacks.TerminateOnNaN(),
                  tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', verbose=1, patience=400, restore_best_weights=True),
                  stopAtLossValue(),
                  tf.keras.callbacks.ModelCheckpoint("M:\\kerascheckpoints", monitor='loss', save_best_only=False,
                 mode='auto', save_freq=5000)])

In [ ]:
estimator.model.summary()

In [ ]:
len(estimator.model.layers)

In [ ]:
estimator.model.layers

In [ ]:
estimator.model.layers

In [ ]:
break


In [ ]:
from keras.utils.vis_utils import plot_model 
plot_model(estimator, to_file='layers_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
ayers = estimator.weights
[ layers[i].name for i in range( len(layers) ) ]

In [ ]:
b

In [ ]:
KerasRegressor(
        build_fn=create_model, verbose=0, learning_rate=0.0001, layers=1, width=1
    ).fit(x_train, x_train,
        epochs=100000,
        batch_size=int(x_train.shape[0]/1),
        #validation_data=(x_test, x_test),
        #validation_data=(x_train, x_train),
       callbacks=[TensorBoard(log_dir='M:\\keras_env'),
                  tf.keras.callbacks.TerminateOnNaN(),
                  tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', verbose=1, patience=400, restore_best_weights=True),
                  stopAtLossValue(),
                  tf.keras.callbacks.ModelCheckpoint("M:\\kerascheckpoints", monitor='loss', save_best_only=False,
                 mode='auto', save_freq=5000)])

In [ ]:
def create_model(
    layers=1,
    kernel_reg=tf.keras.regularizers.l1_l2(l1=0, l2=0),
    width=15,
    learning_rate=0.01,
    dropout_val=0.2,
    input_dim_val=len(X.columns),
):

    # create model
    model = Sequential()
    for i in range(0, layers):
        model.add(
            Dense(
                math.floor(width),
                input_dim=len(X.columns),
                kernel_initializer="normal",
                activation="relu",
                kernel_constraint=maxnorm(4),
                kernel_regularizer=kernel_reg,
            )
        )

    model.add(Dropout(dropout_val))
    model.add(Dense(1, kernel_initializer="normal", activation="relu"))
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)

    metric = tfa.metrics.r_square.RSquare()

    model.compile(
        loss="mean_squared_error",
        optimizer=optimizer,
        metrics=[metric, "mean_squared_error", rmse],
    )

    # print(kernel_reg.get_config())
    return model

In [ ]:
metric_list = ["neg_mean_absolute_error", "r2"]

scores = cross_validate(
        vae,
        X_train,
        X_train,
        scoring=metric_list,
        cv=cv,  # Full CV on final best model
        n_jobs=1,
        verbose=0,
    )

In [ ]:
#pd.DataFrame(df).describe()

In [ ]:
#vae.summary()

In [ ]:
import visualkeras
#visualkeras.layered_view(vae)

In [ ]:
from ann_visualizer.visualize import ann_viz;
#Build your model here
#ann_viz(vae)

In [ ]:
x_train.shape

In [ ]:
vae.history.history['loss'][0]

In [ ]:
vae.history.history['loss'][-1]

In [ ]:
vae.history.history['loss'][-1] / len(x_train)

In [ ]:
decoded_imgs = encoder.predict(x_test, batch_size=x_train.shape[0])

In [ ]:
n = 15
grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

grid_dims = []

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        
        to_append = np.linspace(xi, yi, latent_dim)
        grid_dims.append(to_append)
        #print((xi,yi))

In [ ]:
z_sample = np.linspace(0, 1, latent_dim).reshape(1,-1)
x_decoded = decoder.predict(z_sample)

In [ ]:
if(scale):
    x_decoded = scaler.inverse_transform(x_decoded)
   


In [ ]:
print(len(grid_dims))

In [ ]:
counter = 0
for i in range(0, len(grid_dims), 6):
    
    try:
        z_sample = np.array(grid_dims[i])
        z_sample = z_sample.reshape(1,-1)

        x_decoded = decoder.predict(z_sample)
        x_decoded = scaler.inverse_transform(x_decoded)
        print(grid_dims[i])
        print(x_decoded)
        print(" ")
        div = int(len(mask_array)/3)
        #print(div)
        if(x_decoded[0][0] > 0.1):

            x = x_decoded[0].reshape(div,3)[:,0]
            y = x_decoded[0].reshape(div,3)[:,1]
            z = x_decoded[0].reshape(div,3)[:,2]
            points = np.vstack((x, y, z)).transpose()

            v = pptk.viewer(points)
            #v.set(lookat = (0.15379477,   1.502563  , 115.0956955), phi = 1.57079637, theta=0.09817477, r = 52.53429031)
            v.set(lookat = (6.88162446e-02, 8.74810934e-01, 1.21469368e+02), phi = 1.57079637, theta=-0.11467963, r = 30.0)

            v.capture("M:\\pptk_capture\\" + str(i) +".png")
            time.sleep(2)
            v.close()
    except Exception as e:
        print(e)
        pass

In [ ]:
filenames = os.listdir("M:\\pptk_capture\\")
temp = []
for file in filenames:
    if("png" in file):
        temp.append(int(file.split(".")[0]))
temp.sort()

temp2 = []
for file in temp:
    temp2.append(str(file) + ".png")

In [ ]:
folder_time = datetime.datetime.now().strftime("%Y-%m-%d_%I-%M-%S_%p")
folder_time

In [ ]:
pre = 'M:\\pptk_capture\\'
image_folder='folder_with_images'
fps=2

image_files = temp2.copy()

for i in range(0, len(image_files)):
    image_files[i] = pre + image_files[i]


clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

#clip.write_videofile('M:\\pptk_capture\\vid.webm')
clip.write_videofile("M:\\pptk_archive\\" + folder_time +str(scale_factor) +"_" + str(intermediate_dim)+"_"+ str(deep_dim_one)+"_"+ str(latent_dim)+"_"+str(vae.history.history['loss'][-1]) +".webm")

In [ ]:
image_list = []
#for filename in glob.glob('M:\pptk_capture\*.png'):
for filename in glob.glob('miscimg\*.png'):
    im=Image.open(filename)
    image_list.append(im)

grid_dims = calc_closest_factors(75)
grid = image_grid(image_list, rows=grid_dims[0], cols=grid_dims[1])
folder_time = datetime.datetime.now().strftime("%Y-%m-%d_%I-%M-%S_%p")

im1 = grid.save("M:\\pptk_archive\\" + folder_time +str(scale_factor) +"_" + str(intermediate_dim)+"_"+ str(deep_dim_one)+"_"+ str(latent_dim)+"_"+str(vae.history.history['loss'][-1]) +".png")

In [ ]:
break